# Voting

In [1]:
# Importing the libraries
import numpy as np
import pandas as pd
import sklearn
import matplotlib.pyplot as plt
import sys
import numpy
numpy.set_printoptions(threshold=sys.maxsize)

f_size = 5
# Importing the dataset

# dataset = pd.read_csv('CVD dataset2.csv')
# dataset = pd.read_csv('CVD dataset-plin-4-7-9.csv')
dataset = pd.read_csv('CVD-dataset-2-3-11-13-15.csv')
# dataset = pd.read_csv('CVD-dataset-1-2-3-5-10-11-12-13-14-15.csv')
data=dataset.iloc[:, :].values
X = dataset.iloc[:, :-1].values
y = dataset.iloc[:, f_size].values
# X = dataset.iloc[:, [ 0, 1, 2, 5, 10, 11, 13, 15]].values

rows = len(data)    # 3 rows in your example
cols = len(data[0])
print(rows)
print(cols)

560
6


In [2]:
dataNoCvd = data[~(data[:,f_size] > 0.0)]
dataCvd = data[~(data[:,f_size] < 1.0)]
rowsNoCvd = len(dataNoCvd)    
rowsCvd = len(dataCvd)
print(rowsNoCvd)
print(rowsCvd)
# print(dataNoCvd)
# print(dataCvd)

519
41


In [3]:
def find_NoCvd_limitsArray(i,DownNoCvd, UpNoCvd, fold_NoCvd_total, fold_NoCvd_residue):
    if i>0 :
        DownNoCvd = UpNoCvd
        UpNoCvd= fold_NoCvd_total+UpNoCvd
    if i < fold_NoCvd_residue :
        UpNoCvd+= 1
#     print("NoCvd limits, Down :",DownNoCvd,"Up ",UpNoCvd)
    return   DownNoCvd, UpNoCvd   
 
def find_Cvd_limitsArray(i,DownCvd, UpCvd, fold_Cvd_total, fold_Cvd_residue,cv):
    if i>0 :
        DownCvd = UpCvd
        UpCvd= fold_Cvd_total+UpCvd
    if i >= cv - fold_Cvd_residue :
         UpCvd+= 1     
#     print("Cvd limits, Down :",DownCvd,"Up ",UpCvd)
    return   DownCvd, UpCvd  

def find_testValSubset(DownNoCvd,UpNoCvd,DownCvd,UpCvd):
    temp1=dataNoCvd[DownNoCvd:UpNoCvd,:]
    temp2=dataCvd[DownCvd:UpCvd,:]
    temp3=np.concatenate((temp1, temp2))
#     print(len(temp1),"  ",len(temp2),"  ",len(temp3))
    return temp3

def find_trainSubset(DownNoCvd,UpNoCvd,DownCvd,UpCvd):
    temp1 = np.delete(dataNoCvd, slice(DownNoCvd, UpNoCvd), axis=0)
    temp2 = np.delete(dataCvd, slice(DownCvd, UpCvd), axis=0)
    temp3 = np.concatenate((temp1, temp2))
#     print(len(temp1),"  ",len(temp2),"  ",len(temp3))
    return temp1, temp2, temp3

In [4]:
def create_train_test_sets(cv,dataNoCvd,dataCvd,val_ratio):
    test_total = []
    train_total = []
    train_total_Cvd = []
    train_total_NoCvd = []
    validation_total = []
    
    rowsNoCvd = len(dataNoCvd)    
    rowsCvd = len(dataCvd)

    fold_Cvd_total = rowsCvd//cv
    fold_Cvd_residue= rowsCvd%cv
    print("fold_Cvd_total  :",fold_Cvd_total," fold_Cvd_residue  :",fold_Cvd_residue)

    fold_NoCvd_total = rowsNoCvd//cv
    fold_NoCvd_residue= rowsNoCvd%cv
    print("fold_NoCvd_total:",fold_NoCvd_total,"fold_NoCvd_residue:",fold_NoCvd_residue)
    
    Cvd_val = round(504*val_ratio*(rowsCvd/len(data)))
    noCvd_val = round(504*val_ratio)-Cvd_val
    print(noCvd_val)
    
    DownNoCvd=0
    DownCvd=0
    UpNoCvd = fold_NoCvd_total
    UpCvd = fold_Cvd_total

    for i in range(cv):
        X_test = []
        y_test = []
        X_train = []
        y_train = []
        X_val = []
        y_val = []
#         print("============",i,"==============")
        DownNoCvd, UpNoCvd = find_NoCvd_limitsArray(i, DownNoCvd, UpNoCvd, fold_NoCvd_total, fold_NoCvd_residue)
        DownCvd, UpCvd = find_Cvd_limitsArray(i, DownCvd, UpCvd, fold_Cvd_total, fold_Cvd_residue,cv)
#         print(DownNoCvd,UpNoCvd,DownCvd,UpCvd)
        testSubset_total = find_testValSubset(DownNoCvd,UpNoCvd,DownCvd,UpCvd)
        if i!=9:
#             print(UpNoCvd,UpNoCvd+noCvd_val,UpCvd,UpCvd+Cvd_val)
            validationSubset_total = find_testValSubset(UpNoCvd,UpNoCvd+noCvd_val,UpCvd,UpCvd+Cvd_val)
#             print(DownNoCvd,UpNoCvd+noCvd_val,DownCvd,UpCvd+Cvd_val)
            trainSubset_NoCvd ,trainSubset_Cvd ,trainSubset_total = find_trainSubset(DownNoCvd,UpNoCvd+noCvd_val,DownCvd,UpCvd+Cvd_val)
        else:
#             print(DownNoCvd-noCvd_val,DownNoCvd,DownCvd-Cvd_val,DownCvd)
            validationSubset_total = find_testValSubset(DownNoCvd-noCvd_val,DownNoCvd,DownCvd-Cvd_val,DownCvd)
#             print(DownNoCvd-noCvd_val,UpNoCvd,DownCvd-Cvd_val,UpCvd)
            trainSubset_NoCvd ,trainSubset_Cvd ,trainSubset_total = find_trainSubset(DownNoCvd-noCvd_val,UpNoCvd,DownCvd-Cvd_val,UpCvd)
    #creating X_train, y_train, X_test, y_test
        X_test.append(np.delete(testSubset_total, f_size, axis=1))
        y_test_temp = np.delete(testSubset_total, slice(0, f_size), axis=1)
        y_test.append(np.reshape(y_test_temp, len(y_test_temp)))
        X_test_temp = np.array(X_test)
        X_test = X_test_temp[0]
        y_test_temp = np.array(y_test)
        y_test = y_test_temp[0]
        
        X_val.append(np.delete(validationSubset_total, f_size, axis=1))
        y_val_temp = np.delete(validationSubset_total, slice(0, f_size), axis=1)
        y_val.append(np.reshape(y_val_temp, len(y_val_temp)))
        X_val_temp = np.array(X_val)
        X_val = X_val_temp[0]
        y_val_temp = np.array(y_val)
        y_val = y_val_temp[0]
        
        X_train.append(np.delete(trainSubset_total, f_size, axis=1))
        y_train_temp = np.delete(trainSubset_total, slice(0, f_size), axis=1)
        y_train.append(np.reshape(y_train_temp, len(y_train_temp)))
        X_train_temp = np.array(X_train)
        X_train = X_train_temp[0]
        y_train_temp = np.array(y_train)
        y_train = y_train_temp[0]
        
    #add every subset in a list so we can handle thm later 
        test_total.append([X_test,y_test])
        validation_total.append([X_val,y_val])
        train_total.append([X_train,y_train]) 
        train_total_NoCvd.append(trainSubset_NoCvd)
        train_total_Cvd.append(trainSubset_Cvd)
    return train_total, test_total, validation_total, train_total_NoCvd, train_total_Cvd

In [5]:
train_total, test_total, validation_total, train_total_NoCvd, train_total_Cvd = create_train_test_sets(10,dataNoCvd,dataCvd,0.1)

fold_Cvd_total  : 4  fold_Cvd_residue  : 1
fold_NoCvd_total: 51 fold_NoCvd_residue: 9
46


In [6]:
# Chech ratio in each train and test set
def find_ratio(index_list):
    one = 0
    lenght=len(index_list[0])
    for i in range(lenght):
#         print(index_list[1])
        if index_list[1][i] == 1.0 :
            one+= 1
    ratio = one/lenght
    return ratio

for i in range(10):
    print(len(train_total[i][0]))
    ratio = find_ratio(train_total[i])
    print("ratio in train set:", ratio)
    ratio = find_ratio(test_total[i])
    print("ratio in test set:", ratio)
    ratio = find_ratio(validation_total[i])
    print("ratio in validation set:", ratio)

454
ratio in train set: 0.07268722466960352
ratio in test set: 0.07142857142857142
ratio in validation set: 0.08
454
ratio in train set: 0.07268722466960352
ratio in test set: 0.07142857142857142
ratio in validation set: 0.08
454
ratio in train set: 0.07268722466960352
ratio in test set: 0.07142857142857142
ratio in validation set: 0.08
454
ratio in train set: 0.07268722466960352
ratio in test set: 0.07142857142857142
ratio in validation set: 0.08
454
ratio in train set: 0.07268722466960352
ratio in test set: 0.07142857142857142
ratio in validation set: 0.08
454
ratio in train set: 0.07268722466960352
ratio in test set: 0.07142857142857142
ratio in validation set: 0.08
454
ratio in train set: 0.07268722466960352
ratio in test set: 0.07142857142857142
ratio in validation set: 0.08
454
ratio in train set: 0.07268722466960352
ratio in test set: 0.07142857142857142
ratio in validation set: 0.08
454
ratio in train set: 0.07268722466960352
ratio in test set: 0.07142857142857142
ratio in vali

In [7]:
from xgboost import XGBClassifier


def my_fit(ratio,dataNoCvd,dataCvd, validation_total, params):   
    Subarray=[]
    models = []
    # yarray
    
    rowsNoCvd = len(dataNoCvd)    
    rowsCvd = len(dataCvd)
#     print("Size of NoCvd cases in train set:",rowsNoCvd)
#     print("Size of Cvd cases in train set  :",rowsCvd)


    numOfSubsamples = rowsNoCvd//(rowsCvd*ratio)
    numOfSubsamples= int(numOfSubsamples)
    SubNoCvd = rowsNoCvd//numOfSubsamples
    residue = rowsNoCvd- SubNoCvd*numOfSubsamples


    Up = 0


    for i in range(numOfSubsamples):
        
        classifier = XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=params[5],
                  colsample_bynode=1, colsample_bytree=params[8], gamma=params[4],
                  learning_rate=params[0], max_delta_step=0, max_depth=params[1],
                  min_child_weight=params[3], missing=None, n_estimators=params[2], n_jobs=1,
                  nthread=None, objective='binary:logistic', random_state=0,
                  reg_alpha=params[9], reg_lambda=params[6], scale_pos_weight=params[7], seed=None,
                  silent=None, subsample=params[10], verbosity=1)
    #     print(i)
    #Creating the training set for each model
        Down = Up
        Up= Up + SubNoCvd
        if i < residue :
            Up+= 1
        
#         print(Down)
#         print(Up)
        
        Sub1=dataNoCvd[Down:Up,:]
        Sub2=np.concatenate((Sub1, dataCvd))
        Subarray.append(Sub2)
        
    #Dividing to X and y of the previous traing set    
        X=np.delete(Subarray[i], f_size, axis=1)
        y=np.delete(Subarray[i], slice(0, f_size), axis=1)
        y=np.reshape(y, len(y))
        
        X_val = validation_total[i][0]
        y_val = validation_total[i][1]
        
        eval_set  = [(X,y), (X_val,y_val)]
        
        classifier.fit( X, y, eval_set=eval_set, eval_metric="auc", early_stopping_rounds=50,verbose = False)
#         classifier.fit( X, y)
        
        models.append(classifier)
    return models 

In [8]:
def predict_weighted_average(models,X,weights):
#     print("efwef",len(weights), len(models))
    y_pred = []
    y_pred_models = []
#     weights = [0.16, 0.175, 0.2, 0.147, 0.148, 0.16]
#     weights = [0.15, 0.4, 0.2, 0.05, 0.05, 0.15]
#   weights = [0.16, 0.173, 0.2, 0.147, 0.16, 0.16]
    models_size = len(models)
#     print(models_size)
    X_size = len(X)
            
    for i in range(models_size):
        y_pred_models.append(models[i].predict_proba(X)[:,1]) 
        
    for j in range(X_size):
        sum = 0
        for i in range(models_size):
#             print(i)
#             print(y_pred_models[i][j])
#             print(weights[i])
            sum = sum + y_pred_models[i][j]*weights[i]
        avr = sum
#         /models_size
        if avr>= 0.5 :
            y_pred.append(1.)
        else:
            y_pred.append(0.)
    return y_pred        

In [9]:
# https://towardsdatascience.com/train-test-split-and-cross-validation-in-python-80b61beca4b6

from sklearn.metrics import  confusion_matrix,roc_curve, roc_auc_score, accuracy_score
from imxgboost.imbalance_xgb import imbalance_xgboost as imb_xgb
from sklearn.model_selection import StratifiedKFold # import KFold
from sklearn import metrics

def my_cross_val( cv, train_total, test_total, train_total_NoCvd, train_total_Cvd,params, weights):
    accuracy = []
    specificity = []
    sensitivity = []
    auc = []
    
    # print(X)
    for i in range(cv):
#         X_train = train_total[i][0]
#         y_train = train_total[i][1]
        X_test = test_total[i][0]
        y_test = test_total[i][1]
        


        models = my_fit( 2, train_total_NoCvd[i], train_total_Cvd[i], validation_total,params)


        y_pred = predict_weighted_average(models,X_test,weights)

        cm = confusion_matrix(y_test, y_pred)
    #     print(cm)

        total1=sum(sum(cm))
        #####from confusion matrix calculate accuracy
        accuracy1=(cm[0,0]+cm[1,1])/total1
    #   print ('Accuracy : ', accuracy1)

        specificity1 = cm[0,0]/(cm[0,0]+cm[0,1])
    #   print('Specificity : ', specificity1 )

        sensitivity1 = cm[1,1]/(cm[1,0]+cm[1,1])
    #   print('Sensitivity : ', sensitivity1)
        
#         y = np.array(y_test)
#         pred = np.array(y_pred)
#         fpr, tpr, thresholds = metrics.roc_curve(y, pred, pos_label=2)
#         auc1 = metrics.auc(fpr, tpr)
        
        auc1 = roc_auc_score(y_test, y_pred)

    
        accuracy.append(accuracy1)
        specificity.append(specificity1)
        sensitivity.append(sensitivity1) 
        auc.append(auc1)
    return accuracy, specificity, sensitivity, auc

In [10]:
# https://www.geeksforgeeks.org/python-all-possible-permutations-of-n-lists/

def compute_comb(params):
    
    # initializing lists 
    list1 = params.get("learning_rate")
    list2 = params.get("max_depth")
    list3 = params.get("n_estimators") 
    list4 = params.get("min_child_weight")
    list5 = params.get("gamma")    
    list6 = params.get("colsample_bylevel")
    list7 = params.get("reg_lambda") 
    list8 = params.get("scale_pos_weight")
    list9 = params.get("colsample_bytree")
    list10 = params.get("reg_alpha")
    list11 = params.get("subsample")
    

#     # printing lists  
#     print ("The original lists are : " + str(list1) +
#                                    " " + str(list2) + 
#                                    " " + str(list3) + 
#                                    " " + str(list4) + 
#                                    " " + str(list5) + 
#                                    " " + str(list6) +
#                                    " " + str(list7) +
#                                    " " + str(list8)) 

    # using list comprehension  
    # to compute all possible permutations 
    res = [[i, j, k, l, m, n, o, p,q,r,s] for i in list1  
                                          for j in list2 
                                          for k in list3 
                                          for l in list4 
                                          for m in list5 
                                          for n in list6 
                                          for o in list7
                                          for p in list8
                                          for q in list9
                                          for r in list10
                                          for s in list11] 

    # printing result 
#     print ("All possible permutations are : " +  str(res))
    return res

In [11]:
class style:
    BOLD = '\033[1m'
    END = '\033[0m'

In [24]:
# https://towardsdatascience.com/train-test-split-and-cross-validation-in-python-80b61beca4b6

from sklearn.metrics import  confusion_matrix,roc_curve, roc_auc_score, accuracy_score
from imxgboost.imbalance_xgb import imbalance_xgboost as imb_xgb
from sklearn.model_selection import StratifiedKFold # import KFold
from statistics import mean

def gridsearch(params,weights):
    max_sens = 0
    max_spes = 0
    max_acc = 0
    best_params = []
    max_params = []
    max_auc = 0
    combs = compute_comb(params)
#     print(len(combs))
    for j in range(len(combs)):
#         print(j)
        accuracy, specificity, sensitivity, auc = my_cross_val( 10, train_total, test_total, train_total_NoCvd, train_total_Cvd,combs[j],weights)
        if mean(auc)>0.6:
            print(style.BOLD + "----------------------------------------------------------------"+ style.END)
            print(combs[j])
            print (style.BOLD + 'accuracy    ' + style.END, mean(accuracy))
            print (style.BOLD + 'specificity ' + style.END, mean(specificity))
            print (style.BOLD + 'sensitivity ' + style.END, mean(sensitivity))
            print (style.BOLD + 'auc         ' + style.END, mean(auc))
            print (style.BOLD + 'weights     ' + style.END, weights)
            print(style.BOLD + "----------------------------------------------------------------"+ style.END)
#         else:
#             print(combs[j])
#             print("accuracy    ",mean(accuracy))
#             print("specificity ",mean(specificity))
#             print("sensitivity ",mean(sensitivity))
#             print("auc         ",mean(auc))
#             print("weights     ",weights)
            
        best_params.append([combs[j],mean(accuracy),mean(specificity),mean(sensitivity),mean(auc), weights])
        if mean(auc)> max_auc:
            max_sens = mean(sensitivity)
            max_spes = mean(specificity)
            max_acc = mean(accuracy)
            max_auc = mean(auc)
            max_params = combs[j]
    return max_sens, max_params, max_spes, max_acc, max_auc, best_params

# Ratio = 2

 Αποτέλεσμα της gridsearch με τις ακόλουθες παραμέτρους, γίνονται print όλα και αυτά που ικανοποιούν την συνθήκη : if mean(sensitivity)> 0.5 and mean(specificity)>0.6 and mean(accuracy)>0.6 and mean(auc)>0.5 είναι με bold και μεταξύ γραμμών.

In [30]:
# https://www.analyticsvidhya.com/blog/2016/03/complete-guide-parameter-tuning-xgboost-with-codes-python/        

params = {
                    "learning_rate"    :[0.3],  #0.01-0.2 Makes the model more robust by shrinking the weights on each step
                    "max_depth"        :[2],  #3-10 control over-fitting as higher depth will allow model to learn relations very specific to a particular sample
                    "n_estimators"     :[1000],
                    "min_child_weight" :[3],         #0.5-1 small values might lead to under-fitting
                    "gamma"            :[0],            #Makes the algorithm conservative --> No overfitting
                    "colsample_bylevel" :[0.5], #0.5-1
                     "reg_lambda"      :[1], #  it should be explored to reduce overfitting.
                    "scale_pos_weight" :[4],
                    "colsample_bytree" :[0.75],
                    "reg_alpha"        :[0],
                    "subsample"        :[1.0]
    
        }
# parameters : [0.3, 2, 1000, 3, 0, 0.5, 1, 4, 0.75, 0, 1.0]
# accuracy : 0.6053571428571429
# spes     : 0.5898567119155355
# sens     : 0.81
# auc      : 0.6999283559577677
# [3, 15, 13, 14, 11, 2, 1, 12, 10, 5, 6, 0, 8]
# [3, 15, 13, 14, 11, 2, 1, 12, 10, 5, 6, 0, 8, 4, 7, 9, 16]

In [31]:
total= []
for i in range(5,45,5):
    ii = i/100
    for j in range(5,45,5):
        jj = j/100
        for k in range(5,45,5):
            kk = k/100
            for l in range(5,45,5):
                ll = l/100
                for m in range(5,45,5):
                    mm = m/100
                    for n in range(5,45,5):
                        nn = n/100
                        if ii+jj+kk+ll+mm+nn == 1:
                            weights = [ii, jj, kk, ll, mm, nn]
#                             print(weights)
                            max_sens ,max_params, max_spes, max_acc, max_auc, best_params = gridsearch(params,weights)
                            total.append([max_sens ,max_params, max_spes, max_acc, max_auc, best_params,weights])

----------------------------------------------------------------
[0.3, 2, 1000, 3, 0, 0.5, 1, 4, 0.75, 0, 1.0]
accuracy     0.5928571428571429
specificity  0.5763574660633484
sensitivity  0.81
auc          0.6931787330316742
weights      [0.1, 0.05, 0.1, 0.25, 0.15, 0.35]
----------------------------------------------------------------
----------------------------------------------------------------
[0.3, 2, 1000, 3, 0, 0.5, 1, 4, 0.75, 0, 1.0]
accuracy     0.5946428571428571
specificity  0.5782805429864253
sensitivity  0.81
auc          0.6941402714932127
weights      [0.1, 0.05, 0.15, 0.25, 0.15, 0.3]
----------------------------------------------------------------
----------------------------------------------------------------
[0.3, 2, 1000, 3, 0, 0.5, 1, 4, 0.75, 0, 1.0]
accuracy     0.5892857142857143
specificity  0.5725113122171945
sensitivity  0.81
auc          0.6912556561085973
weights      [0.1, 0.1, 0.05, 0.25, 0.15, 0.35]
---------------------------------------------------

----------------------------------------------------------------
[0.3, 2, 1000, 3, 0, 0.5, 1, 4, 0.75, 0, 1.0]
accuracy     0.5946428571428571
specificity  0.5782805429864254
sensitivity  0.81
auc          0.6941402714932127
weights      [0.15, 0.1, 0.25, 0.15, 0.15, 0.2]
----------------------------------------------------------------
----------------------------------------------------------------
[0.3, 2, 1000, 3, 0, 0.5, 1, 4, 0.75, 0, 1.0]
accuracy     0.5946428571428571
specificity  0.5782805429864254
sensitivity  0.81
auc          0.6941402714932127
weights      [0.15, 0.1, 0.25, 0.2, 0.1, 0.2]
----------------------------------------------------------------
----------------------------------------------------------------
[0.3, 2, 1000, 3, 0, 0.5, 1, 4, 0.75, 0, 1.0]
accuracy     0.5892857142857143
specificity  0.572473604826546
sensitivity  0.81
auc          0.691236802413273
weights      [0.15, 0.15, 0.1, 0.15, 0.15, 0.3]
-------------------------------------------------------

----------------------------------------------------------------
[0.3, 2, 1000, 3, 0, 0.5, 1, 4, 0.75, 0, 1.0]
accuracy     0.6035714285714285
specificity  0.58789592760181
sensitivity  0.81
auc          0.698947963800905
weights      [0.2, 0.05, 0.3, 0.1, 0.15, 0.2]
----------------------------------------------------------------
----------------------------------------------------------------
[0.3, 2, 1000, 3, 0, 0.5, 1, 4, 0.75, 0, 1.0]
accuracy     0.6089285714285715
specificity  0.5955882352941176
sensitivity  0.785
auc          0.6902941176470588
weights      [0.2, 0.1, 0.15, 0.05, 0.25, 0.25]
----------------------------------------------------------------
----------------------------------------------------------------
[0.3, 2, 1000, 3, 0, 0.5, 1, 4, 0.75, 0, 1.0]
accuracy     0.5910714285714286
specificity  0.5743966817496229
sensitivity  0.81
auc          0.6921983408748115
weights      [0.2, 0.1, 0.15, 0.1, 0.15, 0.3]
---------------------------------------------------------

----------------------------------------------------------------
[0.3, 2, 1000, 3, 0, 0.5, 1, 4, 0.75, 0, 1.0]
accuracy     0.6089285714285715
specificity  0.5936274509803922
sensitivity  0.81
auc          0.701813725490196
weights      [0.25, 0.1, 0.2, 0.05, 0.15, 0.25]
----------------------------------------------------------------
----------------------------------------------------------------
[0.3, 2, 1000, 3, 0, 0.5, 1, 4, 0.75, 0, 1.0]
accuracy     0.6053571428571428
specificity  0.5897812971342383
sensitivity  0.81
auc          0.6998906485671191
weights      [0.25, 0.1, 0.25, 0.05, 0.1, 0.25]
----------------------------------------------------------------
----------------------------------------------------------------
[0.3, 2, 1000, 3, 0, 0.5, 1, 4, 0.75, 0, 1.0]
accuracy     0.6142857142857143
specificity  0.5994343891402715
sensitivity  0.81
auc          0.7047171945701358
weights      [0.25, 0.1, 0.25, 0.05, 0.15, 0.2]
----------------------------------------------------

In [33]:
s = 0
for k in range(len(total)):
    for j in range(len(total[k][5])):
        best_params = total[k][5]
        for i in range(len(best_params)):
            if best_params[i][4] > 0.7:
                print("parameters :",best_params[i][0])
                print("accuracy :",best_params[i][1])
                print("spes     :",best_params[i][2])
                print("sens     :",best_params[i][3])
                print("auc      :",best_params[i][4])
                print("weights  :",best_params[i][5])
                print("------------------------------------------------")
                s+=1
print(s)

parameters : [0.3, 2, 1000, 3, 0, 0.5, 1, 4, 0.75, 0, 1.0]
accuracy : 0.6071428571428571
spes     : 0.5917420814479638
sens     : 0.81
auc      : 0.7008710407239819
weights  : [0.2, 0.1, 0.2, 0.1, 0.15, 0.25]
------------------------------------------------
parameters : [0.3, 2, 1000, 3, 0, 0.5, 1, 4, 0.75, 0, 1.0]
accuracy : 0.6125
spes     : 0.5975113122171946
sens     : 0.81
auc      : 0.7037556561085972
weights  : [0.25, 0.05, 0.3, 0.05, 0.15, 0.2]
------------------------------------------------
parameters : [0.3, 2, 1000, 3, 0, 0.5, 1, 4, 0.75, 0, 1.0]
accuracy : 0.6071428571428571
spes     : 0.5917043740573152
sens     : 0.81
auc      : 0.7008521870286576
weights  : [0.25, 0.1, 0.15, 0.05, 0.15, 0.3]
------------------------------------------------
parameters : [0.3, 2, 1000, 3, 0, 0.5, 1, 4, 0.75, 0, 1.0]
accuracy : 0.6089285714285715
spes     : 0.5936274509803922
sens     : 0.81
auc      : 0.701813725490196
weights  : [0.25, 0.1, 0.2, 0.05, 0.15, 0.25]
------------------------

In [14]:
params = {
                    "learning_rate"    :[0.5],  #0.01-0.2 Makes the model more robust by shrinking the weights on each step
                    "max_depth"        :[3],  #3-10 control over-fitting as higher depth will allow model to learn relations very specific to a particular sample
                    "n_estimators"     :[1000],
                    "min_child_weight" :[1],         #0.5-1 small values might lead to under-fitting
                    "gamma"            :[1],            #Makes the algorithm conservative --> No overfitting
                    "colsample_bylevel" :[0.5], #0.5-1
                     "reg_lambda"      :[1], #  it should be explored to reduce overfitting.
                    "scale_pos_weight" :[3],
                    "colsample_bytree" :[0.5],
                    "reg_alpha"        :[0],
                    "subsample"        :[1.0]
    
        }
# parameters : [0.5, 3, 1000, 1, 1, 0.5, 1, 3, 0.5, 0, 1.0]
# accuracy : 0.6589285714285714
# spes     : 0.6532051282051282
# sens     : 0.74
# auc      : 0.6966025641025642
# [13, 11, 15, 2, 3]
# [13, 11, 15, 2, 3, 14, 1, 12, 10, 5, 6, 4, 8, 9, 0, 7, 16]

In [15]:
total= []
for i in range(5,45,5):
    ii = i/100
    for j in range(5,45,5):
        jj = j/100
        for k in range(5,45,5):
            kk = k/100
            for l in range(5,45,5):
                ll = l/100
                for m in range(5,45,5):
                    mm = m/100
                    for n in range(5,45,5):
                        nn = n/100
                        if ii+jj+kk+ll+mm+nn == 1:
                            weights = [ii, jj, kk, ll, mm, nn]
#                             print(weights)
                            max_sens ,max_params, max_spes, max_acc, max_auc, best_params = gridsearch(params,weights)
                            total.append([max_sens ,max_params, max_spes, max_acc, max_auc, best_params,weights])

----------------------------------------------------------------
[0.5, 3, 1000, 1, 1, 0.5, 1, 3, 0.5, 0, 1.0]
accuracy     0.6607142857142857
specificity  0.6551659125188537
sensitivity  0.74
auc          0.6975829562594269
weights      [0.05, 0.1, 0.15, 0.25, 0.25, 0.2]
----------------------------------------------------------------
----------------------------------------------------------------
[0.5, 3, 1000, 1, 1, 0.5, 1, 3, 0.5, 0, 1.0]
accuracy     0.6660714285714285
specificity  0.660972850678733
sensitivity  0.74
auc          0.7004864253393666
weights      [0.05, 0.1, 0.2, 0.25, 0.25, 0.15]
----------------------------------------------------------------
----------------------------------------------------------------
[0.5, 3, 1000, 1, 1, 0.5, 1, 3, 0.5, 0, 1.0]
accuracy     0.6589285714285714
specificity  0.6532805429864253
sensitivity  0.74
auc          0.6966402714932126
weights      [0.05, 0.1, 0.25, 0.25, 0.2, 0.15]
-------------------------------------------------------

----------------------------------------------------------------
[0.5, 3, 1000, 1, 1, 0.5, 1, 3, 0.5, 0, 1.0]
accuracy     0.6696428571428571
specificity  0.6667797888386123
sensitivity  0.715
auc          0.6908898944193061
weights      [0.05, 0.25, 0.05, 0.05, 0.3, 0.3]
----------------------------------------------------------------
----------------------------------------------------------------
[0.5, 3, 1000, 1, 1, 0.5, 1, 3, 0.5, 0, 1.0]
accuracy     0.6767857142857143
specificity  0.6744720965309201
sensitivity  0.715
auc          0.6947360482654601
weights      [0.05, 0.25, 0.1, 0.1, 0.25, 0.25]
----------------------------------------------------------------
----------------------------------------------------------------
[0.5, 3, 1000, 1, 1, 0.5, 1, 3, 0.5, 0, 1.0]
accuracy     0.6714285714285715
specificity  0.6686651583710408
sensitivity  0.715
auc          0.6918325791855203
weights      [0.05, 0.25, 0.1, 0.15, 0.25, 0.2]
---------------------------------------------------

----------------------------------------------------------------
[0.5, 3, 1000, 1, 1, 0.5, 1, 3, 0.5, 0, 1.0]
accuracy     0.6589285714285714
specificity  0.6532051282051282
sensitivity  0.74
auc          0.6966025641025642
weights      [0.1, 0.1, 0.25, 0.2, 0.2, 0.15]
----------------------------------------------------------------
----------------------------------------------------------------
[0.5, 3, 1000, 1, 1, 0.5, 1, 3, 0.5, 0, 1.0]
accuracy     0.6696428571428572
specificity  0.6667420814479638
sensitivity  0.72
auc          0.6933710407239819
weights      [0.1, 0.1, 0.25, 0.25, 0.25, 0.05]
----------------------------------------------------------------
----------------------------------------------------------------
[0.5, 3, 1000, 1, 1, 0.5, 1, 3, 0.5, 0, 1.0]
accuracy     0.6553571428571429
specificity  0.6494343891402715
sensitivity  0.74
auc          0.6947171945701357
weights      [0.1, 0.15, 0.05, 0.2, 0.25, 0.25]
--------------------------------------------------------

----------------------------------------------------------------
[0.5, 3, 1000, 1, 1, 0.5, 1, 3, 0.5, 0, 1.0]
accuracy     0.6785714285714286
specificity  0.6763574660633485
sensitivity  0.715
auc          0.6956787330316742
weights      [0.1, 0.25, 0.1, 0.1, 0.2, 0.25]
----------------------------------------------------------------
----------------------------------------------------------------
[0.5, 3, 1000, 1, 1, 0.5, 1, 3, 0.5, 0, 1.0]
accuracy     0.675
specificity  0.6724736048265461
sensitivity  0.715
auc          0.693736802413273
weights      [0.1, 0.25, 0.15, 0.1, 0.2, 0.2]
----------------------------------------------------------------
----------------------------------------------------------------
[0.5, 3, 1000, 1, 1, 0.5, 1, 3, 0.5, 0, 1.0]
accuracy     0.6696428571428572
specificity  0.6667043740573152
sensitivity  0.715
auc          0.6908521870286577
weights      [0.1, 0.25, 0.15, 0.1, 0.25, 0.15]
----------------------------------------------------------------
----

----------------------------------------------------------------
[0.5, 3, 1000, 1, 1, 0.5, 1, 3, 0.5, 0, 1.0]
accuracy     0.6696428571428572
specificity  0.6667043740573152
sensitivity  0.715
auc          0.6908521870286576
weights      [0.15, 0.1, 0.35, 0.2, 0.15, 0.05]
----------------------------------------------------------------
----------------------------------------------------------------
[0.5, 3, 1000, 1, 1, 0.5, 1, 3, 0.5, 0, 1.0]
accuracy     0.6732142857142858
specificity  0.6705505279034691
sensitivity  0.715
auc          0.6927752639517345
weights      [0.15, 0.1, 0.4, 0.15, 0.15, 0.05]
----------------------------------------------------------------
----------------------------------------------------------------
[0.5, 3, 1000, 1, 1, 0.5, 1, 3, 0.5, 0, 1.0]
accuracy     0.6589285714285714
specificity  0.6532428355957768
sensitivity  0.74
auc          0.6966214177978884
weights      [0.15, 0.15, 0.1, 0.2, 0.2, 0.2]
------------------------------------------------------

----------------------------------------------------------------
[0.5, 3, 1000, 1, 1, 0.5, 1, 3, 0.5, 0, 1.0]
accuracy     0.6482142857142857
specificity  0.6417420814479637
sensitivity  0.74
auc          0.6908710407239819
weights      [0.2, 0.05, 0.1, 0.15, 0.15, 0.35]
----------------------------------------------------------------
----------------------------------------------------------------
[0.5, 3, 1000, 1, 1, 0.5, 1, 3, 0.5, 0, 1.0]
accuracy     0.6482142857142857
specificity  0.6417420814479639
sensitivity  0.74
auc          0.6908710407239819
weights      [0.2, 0.1, 0.05, 0.2, 0.2, 0.25]
----------------------------------------------------------------
----------------------------------------------------------------
[0.5, 3, 1000, 1, 1, 0.5, 1, 3, 0.5, 0, 1.0]
accuracy     0.6482142857142857
specificity  0.6417043740573152
sensitivity  0.74
auc          0.6908521870286576
weights      [0.2, 0.1, 0.1, 0.2, 0.2, 0.2]
------------------------------------------------------------

In [16]:
s = 0
for k in range(len(total)):
    for j in range(len(total[k][5])):
        best_params = total[k][5]
        for i in range(len(best_params)):
            if best_params[i][4] > 0.7:
                print("parameters :",best_params[i][0])
                print("accuracy :",best_params[i][1])
                print("spes     :",best_params[i][2])
                print("sens     :",best_params[i][3])
                print("auc      :",best_params[i][4])
                print("weights  :",best_params[i][5])
                print("------------------------------------------------")
                s+=1
print(s)

parameters : [0.5, 3, 1000, 1, 1, 0.5, 1, 3, 0.5, 0, 1.0]
accuracy : 0.6660714285714285
spes     : 0.660972850678733
sens     : 0.74
auc      : 0.7004864253393666
weights  : [0.05, 0.1, 0.2, 0.25, 0.25, 0.15]
------------------------------------------------
parameters : [0.5, 3, 1000, 1, 1, 0.5, 1, 3, 0.5, 0, 1.0]
accuracy : 0.6678571428571428
spes     : 0.6628582202111614
sens     : 0.74
auc      : 0.7014291101055806
weights  : [0.05, 0.15, 0.25, 0.2, 0.25, 0.1]
------------------------------------------------
parameters : [0.5, 3, 1000, 1, 1, 0.5, 1, 3, 0.5, 0, 1.0]
accuracy : 0.6767857142857143
spes     : 0.6725113122171946
sens     : 0.74
auc      : 0.7062556561085973
weights  : [0.05, 0.15, 0.3, 0.2, 0.25, 0.05]
------------------------------------------------
parameters : [0.5, 3, 1000, 1, 1, 0.5, 1, 3, 0.5, 0, 1.0]
accuracy : 0.6660714285714285
spes     : 0.6608974358974359
sens     : 0.74
auc      : 0.700448717948718
weights  : [0.1, 0.1, 0.2, 0.25, 0.2, 0.15]
-----------------

In [31]:
params = {
                    "learning_rate"    :[0.4],  #0.01-0.2 Makes the model more robust by shrinking the weights on each step
                    "max_depth"        :[3],  #3-10 control over-fitting as higher depth will allow model to learn relations very specific to a particular sample
                    "n_estimators"     :[1000],
                    "min_child_weight" :[1],         #0.5-1 small values might lead to under-fitting
                    "gamma"            :[1],            #Makes the algorithm conservative --> No overfitting
                    "colsample_bylevel" :[0.5], #0.5-1
                     "reg_lambda"      :[1.5], #  it should be explored to reduce overfitting.
                    "scale_pos_weight" :[3],
                    "colsample_bytree" :[0.5],
                    "reg_alpha"        :[0.1],
                    "subsample"        :[1.0]
    
        }
# parameters : [0.4, 3, 1000, 1, 1, 0.5, 1.5, 3, 0.5, 0.1, 1.0]
# accuracy : 0.6482142857142857
# spes     : 0.6417043740573152
# sens     : 0.74
# auc      : 0.6908521870286576
# [13, 15, 11, 2, 3]
# [13, 15, 11, 2, 3, 1, 14, 12, 10, 5, 6, 8, 4, 9, 0, 7, 16]

In [32]:
total= []
for i in range(5,45,5):
    ii = i/100
    for j in range(5,45,5):
        jj = j/100
        for k in range(5,45,5):
            kk = k/100
            for l in range(5,45,5):
                ll = l/100
                for m in range(5,45,5):
                    mm = m/100
                    for n in range(5,45,5):
                        nn = n/100
                        if ii+jj+kk+ll+mm+nn == 1:
                            weights = [ii, jj, kk, ll, mm, nn]
#                             print(weights)
                            max_sens ,max_params, max_spes, max_acc, max_auc, best_params = gridsearch(params,weights)
                            total.append([max_sens ,max_params, max_spes, max_acc, max_auc, best_params,weights])

----------------------------------------------------------------
[0.4, 3, 1000, 1, 1, 0.5, 1.5, 3, 0.5, 0.1, 1.0]
accuracy     0.6446428571428571
specificity  0.6358220211161387
sensitivity  0.765
auc          0.7004110105580694
weights      [0.15, 0.1, 0.4, 0.1, 0.2, 0.05]
----------------------------------------------------------------
----------------------------------------------------------------
[0.4, 3, 1000, 1, 1, 0.5, 1.5, 3, 0.5, 0.1, 1.0]
accuracy     0.6482142857142857
specificity  0.6397435897435897
sensitivity  0.765
auc          0.7023717948717949
weights      [0.15, 0.25, 0.25, 0.1, 0.05, 0.2]
----------------------------------------------------------------
----------------------------------------------------------------
[0.4, 3, 1000, 1, 1, 0.5, 1.5, 3, 0.5, 0.1, 1.0]
accuracy     0.6464285714285714
specificity  0.6378205128205128
sensitivity  0.765
auc          0.7014102564102564
weights      [0.2, 0.15, 0.2, 0.15, 0.1, 0.2]
-------------------------------------------

----------------------------------------------------------------
[0.4, 3, 1000, 1, 1, 0.5, 1.5, 3, 0.5, 0.1, 1.0]
accuracy     0.6392857142857142
specificity  0.6281297134238311
sensitivity  0.79
auc          0.7090648567119155
weights      [0.25, 0.2, 0.15, 0.1, 0.25, 0.05]
----------------------------------------------------------------
----------------------------------------------------------------
[0.4, 3, 1000, 1, 1, 0.5, 1.5, 3, 0.5, 0.1, 1.0]
accuracy     0.6446428571428572
specificity  0.6358974358974359
sensitivity  0.765
auc          0.700448717948718
weights      [0.25, 0.2, 0.15, 0.15, 0.05, 0.2]
----------------------------------------------------------------
----------------------------------------------------------------
[0.4, 3, 1000, 1, 1, 0.5, 1.5, 3, 0.5, 0.1, 1.0]
accuracy     0.6482142857142857
specificity  0.6397435897435898
sensitivity  0.765
auc          0.7023717948717949
weights      [0.25, 0.2, 0.15, 0.15, 0.1, 0.15]
-----------------------------------------

----------------------------------------------------------------
[0.4, 3, 1000, 1, 1, 0.5, 1.5, 3, 0.5, 0.1, 1.0]
accuracy     0.6267857142857143
specificity  0.6146681749622926
sensitivity  0.79
auc          0.7023340874811462
weights      [0.3, 0.1, 0.15, 0.1, 0.15, 0.2]
----------------------------------------------------------------
----------------------------------------------------------------
[0.4, 3, 1000, 1, 1, 0.5, 1.5, 3, 0.5, 0.1, 1.0]
accuracy     0.6357142857142857
specificity  0.6243212669683258
sensitivity  0.79
auc          0.7071606334841629
weights      [0.3, 0.1, 0.15, 0.1, 0.2, 0.15]
----------------------------------------------------------------
----------------------------------------------------------------
[0.4, 3, 1000, 1, 1, 0.5, 1.5, 3, 0.5, 0.1, 1.0]
accuracy     0.6357142857142857
specificity  0.6243589743589744
sensitivity  0.79
auc          0.7071794871794872
weights      [0.3, 0.1, 0.15, 0.15, 0.05, 0.25]
----------------------------------------------

----------------------------------------------------------------
[0.4, 3, 1000, 1, 1, 0.5, 1.5, 3, 0.5, 0.1, 1.0]
accuracy     0.6428571428571428
specificity  0.6320889894419306
sensitivity  0.79
auc          0.7110444947209653
weights      [0.3, 0.2, 0.2, 0.1, 0.05, 0.15]
----------------------------------------------------------------
----------------------------------------------------------------
[0.4, 3, 1000, 1, 1, 0.5, 1.5, 3, 0.5, 0.1, 1.0]
accuracy     0.6339285714285714
specificity  0.6223227752639517
sensitivity  0.79
auc          0.7061613876319759
weights      [0.3, 0.2, 0.2, 0.1, 0.15, 0.05]
----------------------------------------------------------------
----------------------------------------------------------------
[0.4, 3, 1000, 1, 1, 0.5, 1.5, 3, 0.5, 0.1, 1.0]
accuracy     0.6392857142857143
specificity  0.6282051282051282
sensitivity  0.79
auc          0.7091025641025641
weights      [0.3, 0.2, 0.25, 0.1, 0.05, 0.1]
------------------------------------------------

----------------------------------------------------------------
[0.4, 3, 1000, 1, 1, 0.5, 1.5, 3, 0.5, 0.1, 1.0]
accuracy     0.6232142857142857
specificity  0.6108220211161388
sensitivity  0.79
auc          0.7004110105580694
weights      [0.35, 0.15, 0.05, 0.1, 0.2, 0.15]
----------------------------------------------------------------
----------------------------------------------------------------
[0.4, 3, 1000, 1, 1, 0.5, 1.5, 3, 0.5, 0.1, 1.0]
accuracy     0.625
specificity  0.6127450980392157
sensitivity  0.79
auc          0.7013725490196079
weights      [0.35, 0.15, 0.1, 0.05, 0.25, 0.1]
----------------------------------------------------------------
----------------------------------------------------------------
[0.4, 3, 1000, 1, 1, 0.5, 1.5, 3, 0.5, 0.1, 1.0]
accuracy     0.6303571428571428
specificity  0.6185143288084465
sensitivity  0.79
auc          0.7042571644042233
weights      [0.35, 0.15, 0.1, 0.1, 0.15, 0.15]
-------------------------------------------------------

----------------------------------------------------------------
[0.4, 3, 1000, 1, 1, 0.5, 1.5, 3, 0.5, 0.1, 1.0]
accuracy     0.6375
specificity  0.6262443438914027
sensitivity  0.79
auc          0.7081221719457014
weights      [0.4, 0.1, 0.15, 0.15, 0.15, 0.05]
----------------------------------------------------------------
----------------------------------------------------------------
[0.4, 3, 1000, 1, 1, 0.5, 1.5, 3, 0.5, 0.1, 1.0]
accuracy     0.6357142857142857
specificity  0.6243212669683258
sensitivity  0.79
auc          0.7071606334841629
weights      [0.4, 0.15, 0.15, 0.15, 0.1, 0.05]
----------------------------------------------------------------
----------------------------------------------------------------
[0.4, 3, 1000, 1, 1, 0.5, 1.5, 3, 0.5, 0.1, 1.0]
accuracy     0.6285714285714286
specificity  0.6166289592760181
sensitivity  0.79
auc          0.7033144796380091
weights      [0.4, 0.15, 0.2, 0.15, 0.05, 0.05]
------------------------------------------------------

In [33]:
s = 0
for k in range(len(total)):
    for j in range(len(total[k][5])):
        best_params = total[k][5]
        for i in range(len(best_params)):
            if best_params[i][4] > 0.71:
                print("parameters :",best_params[i][0])
                print("accuracy :",best_params[i][1])
                print("spes     :",best_params[i][2])
                print("sens     :",best_params[i][3])
                print("auc      :",best_params[i][4])
                print("weights  :",best_params[i][5])
                print("------------------------------------------------")
                s+=1
print(s)

parameters : [0.4, 3, 1000, 1, 1, 0.5, 1.5, 3, 0.5, 0.1, 1.0]
accuracy : 0.6446428571428571
spes     : 0.6338989441930618
sens     : 0.79
auc      : 0.7119494720965309
weights  : [0.2, 0.25, 0.2, 0.1, 0.15, 0.1]
------------------------------------------------
parameters : [0.4, 3, 1000, 1, 1, 0.5, 1.5, 3, 0.5, 0.1, 1.0]
accuracy : 0.6446428571428572
spes     : 0.6339743589743589
sens     : 0.79
auc      : 0.7119871794871795
weights  : [0.25, 0.15, 0.2, 0.1, 0.1, 0.2]
------------------------------------------------
parameters : [0.4, 3, 1000, 1, 1, 0.5, 1.5, 3, 0.5, 0.1, 1.0]
accuracy : 0.6428571428571429
spes     : 0.632051282051282
sens     : 0.79
auc      : 0.711025641025641
weights  : [0.25, 0.2, 0.15, 0.1, 0.1, 0.2]
------------------------------------------------
parameters : [0.4, 3, 1000, 1, 1, 0.5, 1.5, 3, 0.5, 0.1, 1.0]
accuracy : 0.6410714285714285
spes     : 0.6300904977375565
sens     : 0.79
auc      : 0.7100452488687783
weights  : [0.25, 0.2, 0.15, 0.1, 0.15, 0.15]
-----

In [13]:
# https://www.analyticsvidhya.com/blog/2016/03/complete-guide-parameter-tuning-xgboost-with-codes-python/        

params = {
                    "learning_rate"    :[0.4],  #0.01-0.2 Makes the model more robust by shrinking the weights on each step
                    "max_depth"        :[3],  #3-10 control over-fitting as higher depth will allow model to learn relations very specific to a particular sample
                    "n_estimators"     :[1000],
                    "min_child_weight" :[2],         #0.5-1 small values might lead to under-fitting
                    "gamma"            :[0.5],            #Makes the algorithm conservative --> No overfitting
                    "colsample_bylevel" :[0.5], #0.5-1
                     "reg_lambda"      :[1], #  it should be explored to reduce overfitting.
                    "scale_pos_weight" :[3],
                    "colsample_bytree" :[0.5],
                    "reg_alpha"        :[0.2],
                    "subsample"        :[1.0]
    
        }
# [15, 13, 14, 11, 3, 1, 2, 10, 12, 5]
# [0.4, 3, 1000, 2, 0.5, 0.5, 1, 3, 0.5, 0.2, 1.0]
# 0.6446428571428572
# 0.6360105580693816
# 0.76
# 0.6980052790346908

In [14]:
total= []
for i in range(5,45,5):
    ii = i/100
    for j in range(5,45,5):
        jj = j/100
        for k in range(5,45,5):
            kk = k/100
            for l in range(5,45,5):
                ll = l/100
                for m in range(5,45,5):
                    mm = m/100
                    for n in range(5,45,5):
                        nn = n/100
                        if ii+jj+kk+ll+mm+nn == 1:
                            weights = [ii, jj, kk, ll, mm, nn]
#                             print(weights)
                            max_sens ,max_params, max_spes, max_acc, max_auc, best_params = gridsearch(params,weights)
                            total.append([max_sens ,max_params, max_spes, max_acc, max_auc, best_params,weights])

----------------------------------------------------------------
[0.4, 3, 1000, 2, 0.5, 0.5, 1, 3, 0.5, 0.2, 1.0]
accuracy     0.65
specificity  0.6417043740573153
sensitivity  0.76
auc          0.7008521870286576
weights      [0.05, 0.05, 0.25, 0.2, 0.25, 0.2]
----------------------------------------------------------------
----------------------------------------------------------------
[0.4, 3, 1000, 2, 0.5, 0.5, 1, 3, 0.5, 0.2, 1.0]
accuracy     0.6535714285714286
specificity  0.6455882352941177
sensitivity  0.76
auc          0.7027941176470588
weights      [0.05, 0.1, 0.25, 0.15, 0.25, 0.2]
----------------------------------------------------------------
----------------------------------------------------------------
[0.4, 3, 1000, 2, 0.5, 0.5, 1, 3, 0.5, 0.2, 1.0]
accuracy     0.6517857142857143
specificity  0.6436651583710408
sensitivity  0.76
auc          0.7018325791855203
weights      [0.05, 0.1, 0.3, 0.15, 0.25, 0.15]
--------------------------------------------------------

----------------------------------------------------------------
[0.4, 3, 1000, 2, 0.5, 0.5, 1, 3, 0.5, 0.2, 1.0]
accuracy     0.6553571428571429
specificity  0.6475490196078432
sensitivity  0.76
auc          0.7037745098039215
weights      [0.15, 0.05, 0.25, 0.15, 0.2, 0.2]
----------------------------------------------------------------
----------------------------------------------------------------
[0.4, 3, 1000, 2, 0.5, 0.5, 1, 3, 0.5, 0.2, 1.0]
accuracy     0.6517857142857143
specificity  0.6435897435897435
sensitivity  0.76
auc          0.7017948717948718
weights      [0.15, 0.05, 0.25, 0.15, 0.25, 0.15]
----------------------------------------------------------------
----------------------------------------------------------------
[0.4, 3, 1000, 2, 0.5, 0.5, 1, 3, 0.5, 0.2, 1.0]
accuracy     0.65
specificity  0.6418552036199096
sensitivity  0.76
auc          0.7009276018099547
weights      [0.15, 0.05, 0.25, 0.2, 0.15, 0.2]
------------------------------------------------------

In [16]:
s = 0
for k in range(len(total)):
    for j in range(len(total[k][5])):
        best_params = total[k][5]
        for i in range(len(best_params)):
            if best_params[i][4] > 0.705:
                print("parameters :",best_params[i][0])
                print("accuracy :",best_params[i][1])
                print("spes     :",best_params[i][2])
                print("sens     :",best_params[i][3])
                print("auc      :",best_params[i][4])
                print("weights  :",best_params[i][5])
                print("------------------------------------------------")
                s+=1
print(s)

parameters : [0.4, 3, 1000, 2, 0.5, 0.5, 1, 3, 0.5, 0.2, 1.0]
accuracy : 0.6589285714285714
spes     : 0.6513197586726999
sens     : 0.76
auc      : 0.7056598793363499
weights  : [0.15, 0.1, 0.2, 0.1, 0.2, 0.25]
------------------------------------------------
1


In [31]:
# https://www.analyticsvidhya.com/blog/2016/03/complete-guide-parameter-tuning-xgboost-with-codes-python/        

params = {
                    "learning_rate"    :[0.5],  #0.01-0.2 Makes the model more robust by shrinking the weights on each step
                    "max_depth"        :[3],  #3-10 control over-fitting as higher depth will allow model to learn relations very specific to a particular sample
                    "n_estimators"     :[1000],
                    "min_child_weight" :[1],         #0.5-1 small values might lead to under-fitting
                    "gamma"            :[1],            #Makes the algorithm conservative --> No overfitting
                    "colsample_bylevel" :[0.75], #0.5-1
                     "reg_lambda"      :[1], #  it should be explored to reduce overfitting.
                    "scale_pos_weight" :[3],
                    "colsample_bytree" :[0.5],
                    "reg_alpha"        :[0.2],
                    "subsample"        :[1.0]
    
        }
# parameters : [0.5, 3, 1000, 1, 1, 0.75, 1, 3, 0.5, 0.2, 1.0]
# accuracy : 0.6482142857142857
# spes     : 0.641628959276018
# sens     : 0.74
# auc      : 0.690814479638009
# [13, 2, 3, 15, 11]
# [13, 2, 3, 15, 11, 1, 14, 10, 12, 5, 6, 8, 4, 0, 7, 9, 16]

In [32]:
total= []
for i in range(5,45,5):
    ii = i/100
    for j in range(5,45,5):
        jj = j/100
        for k in range(5,45,5):
            kk = k/100
            for l in range(5,45,5):
                ll = l/100
                for m in range(5,45,5):
                    mm = m/100
                    for n in range(5,45,5):
                        nn = n/100
                        if ii+jj+kk+ll+mm+nn == 1:
                            weights = [ii, jj, kk, ll, mm, nn]
#                             print(weights)
                            max_sens ,max_params, max_spes, max_acc, max_auc, best_params = gridsearch(params,weights)
                            total.append([max_sens ,max_params, max_spes, max_acc, max_auc, best_params,weights])

In [37]:
s = 0
for k in range(len(total)):
    for j in range(len(total[k][5])):
        best_params = total[k][5]
        for i in range(len(best_params)):
            if best_params[i][4] > 0.7:
                print("parameters :",best_params[i][0])
                print("accuracy :",best_params[i][1])
                print("spes     :",best_params[i][2])
                print("sens     :",best_params[i][3])
                print("auc      :",best_params[i][4])
                print("weights  :",best_params[i][5])
                print("------------------------------------------------")
                s+=1
print(s)

0


In [26]:
# https://www.analyticsvidhya.com/blog/2016/03/complete-guide-parameter-tuning-xgboost-with-codes-python/        

params = {
                    "learning_rate"    :[0.4],  #0.01-0.2 Makes the model more robust by shrinking the weights on each step
                    "max_depth"        :[3],  #3-10 control over-fitting as higher depth will allow model to learn relations very specific to a particular sample
                    "n_estimators"     :[1000],
                    "min_child_weight" :[1],         #0.5-1 small values might lead to under-fitting
                    "gamma"            :[1],            #Makes the algorithm conservative --> No overfitting
                    "colsample_bylevel" :[0.5], #0.5-1
                     "reg_lambda"      :[1.5], #  it should be explored to reduce overfitting.
                    "scale_pos_weight" :[3],
                    "colsample_bytree" :[0.5],
                    "reg_alpha"        :[0.1],
                    "subsample"        :[1.0]
    
        }

In [38]:
weights = [0.27, 0.2, 0.17, 0.1, 0.11, 0.15]

max_sens ,max_params, max_spes, max_acc, max_auc, best_params = gridsearch(params,weights)

# parameters : [0.4, 3, 1000, 1, 1, 0.5, 1.5, 3, 0.5, 0.1, 1.0]
# accuracy : 0.6464285714285715
# spes     : 0.6358974358974359
# sens     : 0.79
# auc      : 0.712948717948718
# weights  : [0.25, 0.2, 0.2, 0.1, 0.1, 0.15]

----------------------------------------------------------------
[0.4, 3, 1000, 1, 1, 0.5, 1.5, 3, 0.5, 0.1, 1.0]
accuracy     0.6482142857142857
specificity  0.6378205128205128
sensitivity  0.79
auc          0.7139102564102564
weights      [0.27, 0.2, 0.17, 0.1, 0.11, 0.15]
----------------------------------------------------------------


In [ ]:
[0.4, 3, 1000, 1, 1, 0.5, 1.5, 3, 0.5, 0.1, 1.0]
accuracy     0.6482142857142857
specificity  0.6378205128205128
sensitivity  0.79
auc          0.7139102564102564
weights      [0.27, 0.2, 0.18, 0.1, 0.1, 0.15]

[0.4, 3, 1000, 1, 1, 0.5, 1.5, 3, 0.5, 0.1, 1.0]
accuracy     0.6482142857142857
specificity  0.6378205128205128
sensitivity  0.79
auc          0.7139102564102564
weights      [0.27, 0.2, 0.17, 0.1, 0.11, 0.15]

In [ ]:
parameters : [0.5, 3, 1000, 1, 1, 0.5, 1, 3, 0.5, 0, 1.0]
accuracy : 0.6767857142857143
spes     : 0.6725113122171946
sens     : 0.74
auc      : 0.7062556561085973
weights  : [0.05, 0.15, 0.3, 0.2, 0.25, 0.05]

parameters : [0.4, 3, 1000, 1, 1, 0.5, 1.5, 3, 0.5, 0.1, 1.0]
accuracy : 0.6446428571428571
spes     : 0.6338989441930618
sens     : 0.79
auc      : 0.7119494720965309
weights  : [0.2, 0.25, 0.2, 0.1, 0.15, 0.1]

parameters : [0.4, 3, 1000, 1, 1, 0.5, 1.5, 3, 0.5, 0.1, 1.0]
accuracy : 0.6464285714285715
spes     : 0.6358974358974359
sens     : 0.79
auc      : 0.712948717948718
weights  : [0.25, 0.2, 0.2, 0.1, 0.1, 0.15]

parameters : [0.4, 3, 1000, 1, 1, 0.5, 1.5, 3, 0.5, 0.1, 1.0]
accuracy : 0.6410714285714285
spes     : 0.6300904977375565
sens     : 0.79
auc      : 0.7100452488687783
weights  : [0.3, 0.25, 0.25, 0.1, 0.05, 0.05]

